# 1.Graph classification task
# Dataset: Molecular_Sample.csv

### 1.1. Extract graph features with eigenvector centrality, Katz centrality, Laplacian centrality
### 1.2.Train model (SVM/MLP) with graph features as concatenation of above graph features,  with test_size = 0.3

# 2. Graph classification task
# Dataset: Molecular_Sample.csv, load p_np column as label (y)

### Train model (SVM/MLP) with graph features as graphlet kernel (X) from practice code W7 with test_size = 0.3, with graphlets kernel size = 3

In [3]:
# Graphlet Kernel
import networkx as nx
import numpy as np
from itertools import combinations

def compute_graphlet_counts(G, k):
    """
    Compute the counts of k-node graphlets in the given graph.

    Parameters:
        G (networkx.Graph): The input graph.
        k (int): The size of the graphlets to count.

    Returns:
        dict: A dictionary containing the counts of k-node graphlets.
    """
    graphlet_counts = {graphlet: 0 for graphlet in range(1, k+1)}

    for node in G.nodes():
    # Generate the induced subgraph with node and its neighbors
        subgraph_nodes = [node] + list(G.neighbors(node))
        subgraph = G.subgraph(subgraph_nodes)

        # Count the occurrences of k-node graphlets in the subgraph
        for graphlet_size in range(1, k+1):
            for subgraph_nodes_subset in combinations(subgraph_nodes, graphlet_size):
                if G.subgraph(subgraph_nodes_subset).number_of_nodes() == graphlet_size:
                    graphlet_counts[graphlet_size] += 1

    return graphlet_counts #Use graphlet_counts as feature vector

#Change molecule to graph
def mol_to_nx_graph(mol):
    """
    Convert RDKit molecule to NetworkX graph.

    Parameters:
        mol (RDKit molecule): The input molecule.

    Returns:
        networkx.Graph: The NetworkX graph representation of the molecule.
    """
    G = nx.Graph()

    # Add nodes for atoms
    for atom in mol.GetAtoms():
        G.add_node(atom.GetIdx(), atomic_num=atom.GetAtomicNum())

    # Add edges for bonds
    for bond in mol.GetBonds():
        G.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), bond_type=bond.GetBondType())

    return G

# Try to convert molecules to NetworkX graphs
# Compute the graph kernel for each molecule
# After that, we can train a machine learning model


# 3. Graph classification task
# Dataset: Molecular_Sample.csv, load p_np column as label (y)

### Train model (SVM/MLP) with graph features as random walk kernel (X) from practice code W7 with test_size = 0.3, walk_length = 4, num_walks = 100

In [4]:
import random

def random_walk(graph, walk_length):
    """
    Perform a random walk of a given length on a graph starting from a random node.

    Parameters:
        graph (networkx.Graph): The input graph.
        walk_length (int): The length of the random walk.

    Returns:
        tuple: The sequence of nodes in the random walk.
    """
    node = random.choice(list(graph.nodes()))
    walk = [node]
    for _ in range(walk_length - 1):
        neighbors = list(graph.neighbors(node))
        if neighbors:
            node = random.choice(neighbors)
            walk.append(node)
        else:
            break

    return tuple(walk)

def random_walk_self_kernel(graph, walk_length=2, num_walks=20):
    """
    Calculate the random walk kernel for a single graph.

    Parameters:
        graph (networkx.Graph): The input graph.
        walk_length (int): The length of the random walk.
        num_walks (int): The number of random walks to perform.

    Returns:
        float: The normalized random walk kernel value for the graph.
    """
    walk_counts = {}

    for _ in range(num_walks):
        walk = random_walk(graph, walk_length)
        if walk in walk_counts:
            walk_counts[walk] += 1
        else:
            walk_counts[walk] = 1

    # Calculate the kernel value based on the counts of identical walks
    kernel_value = sum(count ** 2 for count in walk_counts.values())
    
    # Normalize the kernel value
    kernel_value /= num_walks

    return kernel_value

# Same with the graphlet kernel instruction

# 4. K-means Clustering task
# Dataset: Molecular_Sample.csv

### Perfrom k-means clustering with Weisfeiler-Lehman hashes, number of cluster = 2

In [5]:
# Functions for hashing graphs to strings
"""
Functions for hashing graphs to strings.
Isomorphic graphs should be assigned identical hashes.
For now, only Weisfeiler-Lehman hashing is implemented.
"""
from hashlib import blake2b

def _hash_label(label, digest_size):
    return blake2b(label.encode("ascii"), digest_size=digest_size).hexdigest()

def _init_node_labels(G, edge_attr, node_attr):
    if node_attr:
        return {u: str(dd[node_attr]) for u, dd in G.nodes(data=True)}
    elif edge_attr:
        return {u: "" for u in G}
    else:
        return {u: str(deg) for u, deg in G.degree()}

def _neighborhood_aggregate(G, node, node_labels, edge_attr=None):
    """
    Compute new labels for given node by aggregating
    the labels of each node's neighbors.
    """
    label_list = []
    for nbr in G.neighbors(node):
        prefix = "" if edge_attr is None else str(G[node][nbr][edge_attr])
        label_list.append(prefix + node_labels[nbr])
    return node_labels[node] + "".join(sorted(label_list))

def weisfeiler_lehman_graph_hash(
    G, edge_attr=None, node_attr=None, iterations=3, digest_size=16
):
    def weisfeiler_lehman_step(G, labels, edge_attr=None):
        """
        Apply neighborhood aggregation to each node
        in the graph.
        Computes a dictionary with labels for each node.
        """
        new_labels = {}
        for node in G.nodes():
            label = _neighborhood_aggregate(G, node, labels, edge_attr=edge_attr)
            new_labels[node] = _hash_label(label, digest_size)
        return new_labels

    # set initial node labels
    node_labels = _init_node_labels(G, edge_attr, node_attr)

    subgraph_hash_counts = []
    for _ in range(iterations):
        node_labels = weisfeiler_lehman_step(G, node_labels, edge_attr=edge_attr)
        counter = Counter(node_labels.values())
        # sort the counter, extend total counts
        subgraph_hash_counts.extend(sorted(counter.items(), key=lambda x: x[0]))

    # hash the final counter
    return _hash_label(str(tuple(subgraph_hash_counts)), digest_size)

# Define a function to convert a SMILES string to a NetworkX graph
# Apply Weisfeiler-Lehman hashing to each molecule graph
# Convert molecules to NetworkX graphs
# Calculate Weisfeiler-Lehman hash for each molecule graph
# Convert Weisfeiler-Lehman hashes to a numerical representation
# Perform K-means clustering
from sklearn.cluster import KMeans
n_clusters = 2  #number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# cluster_labels = kmeans.fit_predict(X_numerical)